In [1]:
import xarray as xr
from pathlib import Path

data_root = Path('data/neodc/esacci/soil_moisture/data/daily_files/ACTIVE/v08.1/2022/')

True

In [3]:
cci_ssm_datacube = xr.open_mfdataset(data_root.glob("*.nc"))
cci_ssm_datacube

<xarray.Dataset> Size: 2GB
Dimensions:         (time: 62, lat: 720, lon: 1440)
Coordinates:
  * time            (time) datetime64[ns] 496B 2022-07-01 ... 2022-08-31
  * lat             (lat) float64 6kB 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
  * lon             (lon) float64 12kB -179.9 -179.6 -179.4 ... 179.6 179.9
Data variables:
    sm              (time, lat, lon) float32 257MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sm_uncertainty  (time, lat, lon) float32 257MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    flag            (time, lat, lon) float32 257MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    freqbandID      (time, lat, lon) int16 129MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    dnflag          (time, lat, lon) int8 64MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    mode            (time, lat, lon) int8 64MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sensor          (time, lat, lon) int32 257MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    t0              (time, lat, lon) datetime64[ns] 514MB dask.array<chunksize=(1, 360, 720), meta=np.ndarray>
Attributes: (12/44)
    title:                        ESA CCI Surface Soil Moisture merged ACTIVE...
    institution:                  TU Wien (AUT); VanderSat B.V. (NL), Planet ...
    contact:                      cci_sm_contact@eodc.eu
    source:                       WARP 5.5R1.1/AMI-WS/ERS12 Level 2 Soil Mois...
    platform:                     ERS-1, ERS-2, METOP-A, METOP-B
    processing_level:             Quality-controlled, super-collocated (L3S) ...
    ...                           ...
    geospatial_lon_units:         degrees_east
    geospatial_lat_resolution:    0.25 degree
    geospatial_lon_resolution:    0.25 degree
    spatial_resolution:           25km
    time_coverage_start:          20220701T000000Z
    time_coverage_end:            20220701T235959Z

In [10]:
ssm_datacube = cci_ssm_datacube['sm']
ssm_datacube

<xarray.DataArray 'sm' (time: 62, lat: 720, lon: 1440)> Size: 257MB
dask.array<concatenate, shape=(62, 720, 1440), dtype=float32, chunksize=(1, 720, 1440), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 496B 2022-07-01 2022-07-02 ... 2022-08-31
  * lat      (lat) float64 6kB 89.88 89.62 89.38 89.12 ... -89.38 -89.62 -89.88
  * lon      (lon) float64 12kB -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
Attributes:
    _CoordinateAxes:      time lat lon
    ancillary_variables:  sm_uncertainty flag t0
    dtype:                float32
    long_name:            Percent of Saturation Soil Moisture
    units:                percent
    valid_range:          [  0. 100.]